In [3]:
import http.client

import json

def obter_endereco_por_cep(cep):
    conexao = http.client.HTTPConnection("viacep.com.br")
    conexao.request("GET", f"/ws/{cep}/json/")
    resposta = conexao.getresponse()
    dados = resposta.read()
    endereco = json.loads(dados.decode("utf-8"))
    conexao.close()

    if "erro" not in endereco:
        return endereco
    else:
        return "CEP não encontrado."

cep_exemplo = "53429080"

endereco_resultado = obter_endereco_por_cep(cep_exemplo)

print(endereco_resultado)

{'cep': '53429-080', 'logradouro': 'Rua Chile', 'complemento': '', 'unidade': '', 'bairro': 'Nossa Senhora do Ó', 'localidade': 'Paulista', 'uf': 'PE', 'ibge': '2610707', 'gia': '', 'ddd': '81', 'siafi': '2513'}


In [ ]:
pip install pandas openpyxl

In [5]:
import http.client
import json
import pandas as pd

def obter_endereco_por_cep(cep):
    conexao = http.client.HTTPConnection("viacep.com.br")
    conexao.request("GET", f"/ws/{cep}/json/")
    resposta = conexao.getresponse()
    dados = resposta.read()
    endereco = json.loads(dados.decode("utf-8"))
    conexao.close()

    if "erro" not in endereco:
        return endereco
    else:
        return "CEP não encontrado."

cep_exemplo = "53429080"

endereco_resultado = obter_endereco_por_cep(cep_exemplo)

def salvar_endereco_excel(endereco, nome_arquivo="endereco.xlsx"):
    if "erro" not in endereco:
        df = pd.DataFrame([endereco])
        df.to_excel(nome_arquivo, index=False)
        print(f"Dados salvos com sucesso no arquivo {nome_arquivo}.")
    
    else:
        print(f"Não foi possível salvar os dados: CEP não encontrado.")

salvar_endereco_excel(endereco_resultado)

Dados salvos com sucesso no arquivo endereco.xlsx.


In [17]:
import http.client
import json
import pandas as pd

def obter_endereco_por_cep(cep):
    conexao = http.client.HTTPConnection("viacep.com.br")
    conexao.request("GET", f"/ws/{cep}/json/")
    resposta = conexao.getresponse()

    if resposta.status != 200:
        conexao.close()
        return None
    
    dados = resposta.read()
    endereco = json.loads(dados.decode("utf-8"))
    conexao.close()

    return endereco if "erro" not in endereco else None


caminho_planilha =  "CEP.xlsx"

planilha_ceps = pd.read_excel(caminho_planilha,sheet_name='CEP')

ceps = planilha_ceps['CEP'].dropna()

resultados = pd.DataFrame(columns=['CEP','Logradouro','Bairro','Localidade','UF'])

for cep in ceps:
    endereco = obter_endereco_por_cep(str(cep).replace('-',''))
    if endereco:
        nova_linha = pd.DataFrame([{
            'CEP':  cep,
            'Logradouro': endereco.get('logradouro',''),
            'Bairro': endereco.get('bairro',''),
            'Localidade': endereco.get('localidade',''),
            'UF': endereco.get('uf','')
        }])
        resultados = pd.concat([resultados, nova_linha], ignore_index=True)

with pd.ExcelWriter(caminho_planilha, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    resultados.to_excel(writer, sheet_name='Dados', index=False)

print("Endereços salvos na aba 'Dados' na planilha 'CEP.xlsx'")

Endereços salvos na aba 'Dados' na planilha 'CEP.xlsx'
